In [1]:
#!connect signalr --kernel-name rover --hub-url http://10.0.0.252:1024/kernelhub

In [1]:
#!rover
resourceScanner.CaptureFromCamera(roverBody.Camera);
var sign = 1;

In [1]:
#!rover
using PiTop.Interactive.Rover.ImageProcessing;
using System.Threading;
using System.IO;
resourceScanner.LoadModel(new DirectoryInfo("/home/pi/dotnet_conf_resources/models/v5"));

In [1]:
#!rover
var frame = roverBody.Camera.GetFrame().Preview();
var lastScanResult = resourceScanner.AnalyseFrame(frame);

In [1]:
#!rover
lastScanResult = null;
roverBody.AllLightsOff();

roverBrain.Perceive = (rover, now, token) => {

};

roverBrain.Plan = (rover, now, token) => {
    Task.Delay(2000).Wait();
    return PlanningResult.NoPlan;
};

roverBrain.Act = (rover, now, token) => { 
    
};

In [1]:
#!rover
//water
roverBody.TiltController.Pan = Angle.FromDegrees(50);
roverBody.TiltController.Tilt = Angle.FromDegrees(30);
await Task.Delay(1000);
frame = roverBody.Camera.GetFrame();
display(frame.Preview());
lastScanResult = resourceScanner.AnalyseFrame(frame);
display(lastScanResult?.Classification);

In [1]:
#!rover
//lightning
roverBody.TiltController.Pan = Angle.FromDegrees(-90);
roverBody.TiltController.Tilt = Angle.FromDegrees(35);
await Task.Delay(1000);
frame = roverBody.Camera.GetFrame();
display(frame.Preview());
lastScanResult = resourceScanner.AnalyseFrame(frame);
display(lastScanResult?.Classification);

In [1]:
#!rover
//rainbow
roverBody.TiltController.Pan = Angle.FromDegrees(-50);
roverBody.TiltController.Tilt = Angle.FromDegrees(-5);
await Task.Delay(1000);
frame = roverBody.Camera.GetFrame();
display(frame.Preview());
lastScanResult = resourceScanner.AnalyseFrame(frame);
display(lastScanResult?.Classification);

In [1]:
#!rover
//grass
roverBody.TiltController.Pan = Angle.FromDegrees(25);
roverBody.TiltController.Tilt = Angle.FromDegrees(-20);
await Task.Delay(1000);
frame = roverBody.Camera.GetFrame();
display(frame.Preview());
lastScanResult = resourceScanner.AnalyseFrame(frame);
display(lastScanResult?.Classification);


In [1]:
#!rover
display(frame.Preview());
display(lastScanResult.Classification);
display(lastScanResult.Classifications);